In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
data = pd.read_csv(r"C:\Users\bharg\OneDrive\Desktop\PS - 1\blizzard_final_complete_hourly_data_2016_2019.csv")
data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
X = data[['tempr', 'ap', 'ws', 'rh']]
y = data['blizzard']

In [ ]:
# Reshape the input data for a CNN model
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)
X_scaled = scaler.transform(X.reshape(-1, 1)).reshape(X.shape)

In [ ]:
class_weights = {0:1, 1: 5}

In [ ]:
# Create the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1, class_weight=class_weights)

In [ ]:
# Make predictions on the test data
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

In [ ]:
blizzard_predicted_prob =  model.predict(X_scaled)
blizzard_predicted = (blizzard_predicted_prob > 0.5).astype(int)

In [ ]:
data['blizzard_predicted'] = blizzard_predicted
data

In [ ]:
conf_matrix = confusion_matrix(data['blizzard'], data['blizzard_predicted'])
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
data['is_it'] = ['1' if data['blizzard'][i] == 1 and data['blizzard_predicted'][i] == 1 else '2' if data['blizzard'][i] == 0 and data['blizzard_predicted'][i] == 1 else '3' if data['blizzard'][i] == 1 and data['blizzard_predicted'][i] == 0 else 0 for i in range(len(data))]
data

In [ ]:
plt.scatter(data['date'],data['is_it'])

In [ ]:
plt.figure(figsize=(17, 6))
plt.plot(data['date'][-8904: ],data['ws'][-8904: ], linewidth = 0.5)
for i in range(len(data)-8904,len(data)):
    if data['is_it'][i] == '1':
        plt.scatter(data['date'][i], data['ws'][i], s=20 ,c='green')
    elif data['is_it'][i] == '2':
        plt.scatter(data['date'][i], data['ws'][i],s=20 ,c='yellow')
    elif data['is_it'][i] == '3':
        plt.scatter(data['date'][i], data['ws'][i], s=20,c='red')
plt.title("Classification of Blizzard using CNN\n Duration: 2020-01-29 to 2021-02-04\n Training Duration: 2016-01-01 to 2020-01-29", fontweight='bold', fontsize = 10)
plt.xlabel('Date')
plt.ylabel('Wind speed (m/s)')
pop_a = mpatches.Patch(color='red', label='Missed')
pop_b = mpatches.Patch(color='yellow', label='Incorrect')
pop_c = mpatches.Patch(color='green', label='Correct')
pop_d = mpatches.Patch(color='blue', label='Wind Speed')
plt.legend(handles=[pop_a,pop_b, pop_c, pop_d])
plt.savefig("Blizzard.png", format='png', dpi=800)
